In [2]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
import scipy
from scipy import stats
import matplotlib.pyplot as plt
import torch
#from sklearn.pipeline import make_pipeline
#from sklearn.compose import make_column_transformer

In [3]:
pd.set_option('display.max_colwidth', None)

### Data Observations

##### More than half the dataset is from the NucleaSeq Finkelstein experiments
##### Finkelstein experiments do not contain cell line or chromosome due to being in vitro
##### Finkelstein experiments do not contain target_context
##### Sequences are variable lengths
##### target_sequence ranges in length 20-25 nt
##### grna_target_sequence ranges in length 22-24 nt
##### 75 pairs which do not have a cleavage frequency
##### Length of grna != length of target for some pairs

### Split Training and Testing

### Preprocessing

In [4]:
class Preprocessing:
    def select(df, params):
        df = df[params].convert_dtypes()
        return df
    
    def drop_na(df, params):
        for col in params:
            df = df[df[col].notna()]
        return df
    
    def remove_dash(df, params):
        for col in df.select_dtypes(exclude=['number']).columns:
            df[col] = [
                seq.replace("-", "")
                for seq in df[col]
            ]
        return df
    
    def pad(df, params):
        for col in df.select_dtypes(exclude=['number']).columns:
            df[col] = df[col].str.pad(width=50, side='right', fillchar='X')
        return df
    
    def encode_nt(nt: str) -> int:
        assert len(nt) == 1
        encoding_dict = {
            'X':0, 
            'A':0.25,
            'T':0.50,
            'G':0.75,
            'C':1.00
        }
        return encoding_dict.get(nt.upper())
    
    def encode_seq(seq: str):
        encoding = [
            Preprocessing.encode_nt(nt)
            for nt in seq
        ]
        return np.array(encoding)
    
    def encode(df, params):
        for col in df.select_dtypes(exclude=['number']).columns:
            df[col] = [
                Preprocessing.encode_seq(seq)
                for seq in df[col]
            ]
        return df
    
    def fold_seq(df):
        df["stacked"] = df["grna_target_sequence"].apply(lambda x: x.tolist()) + df["target_sequence"].apply(lambda x: x.tolist())
        df["stacked"] = df["stacked"].apply(lambda x: np.array(x))
        return df["stacked"]

    def tensorfy(stacked):
        temp = []
        for i in stacked:
            temp.append(i)
        return torch.from_numpy(np.array(temp).astype(np.float32))
    
    def preprocess(df, params):
        processed_df = Preprocessing.select(df, params)
        processed_df = Preprocessing.drop_na(processed_df, params)
        processed_df = Preprocessing.remove_dash(processed_df, params)
        processed_df = Preprocessing.pad(processed_df, params)
        processed_df = Preprocessing.encode(processed_df, params)
        stacked = Preprocessing.fold_seq(processed_df)
        X = Preprocessing.tensorfy(stacked)
        return X

##### Version 1

In [7]:
df_train = pd.read_csv("../data/train.csv")

In [6]:
params = ["grna_target_sequence", "target_sequence"]

In [9]:
def get_X(df, params):
    df_X = df.drop(columns = ["cleavage_freq"])
    X = Preprocessing.preprocess(df_X, params)
    return X

In [11]:
df_X = df_train.drop(columns = ["cleavage_freq"])
df_X.shape

(20505, 29)

In [8]:
X = Preprocessing.preprocess(df_X, params)
X

tensor([[0.7500, 0.2500, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.7500, 0.7500, 0.7500,  ..., 0.0000, 0.0000, 0.0000],
        [0.7500, 0.5000, 0.7500,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.7500, 0.5000, 0.7500,  ..., 0.0000, 0.0000, 0.0000],
        [0.7500, 0.5000, 0.7500,  ..., 0.0000, 0.0000, 0.0000],
        [0.7500, 0.5000, 0.7500,  ..., 0.0000, 0.0000, 0.0000]])

In [10]:
X = get_X(df_train, params)
print(X)

tensor([[0.7500, 0.2500, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.7500, 0.7500, 0.7500,  ..., 0.0000, 0.0000, 0.0000],
        [0.7500, 0.5000, 0.7500,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.7500, 0.5000, 0.7500,  ..., 0.0000, 0.0000, 0.0000],
        [0.7500, 0.5000, 0.7500,  ..., 0.0000, 0.0000, 0.0000],
        [0.7500, 0.5000, 0.7500,  ..., 0.0000, 0.0000, 0.0000]])


In [ ]:
fork

### Data Transformation

##### According to reference, should transform cleavage rates to approximate a Gaussian with zero mean and variance 𝜎^2=2 using the nonlinear Box–Cox transformation.

##### Values above and below ±2𝜎 should be capped in order to achieve a fixed value range
##### measured cleavage rates below the lowest reported measurement accuracy (10−5 in our case) should be manually transformed to −2𝜎

In [ ]:
# Separate out cleavage rates < 10^-5
low_cleavage = df_train[df_train["cleavage_freq"] < 10**-5]
df_train = df_train[df_train["cleavage_freq"]>= 10**-5]

df_train["transformed_cleavage_freq"], _ = stats.boxcox(df_train["cleavage_freq"])

# Scale ** Surely there's an easier way to do this?
m1 = scipy.mean(df_train["transformed_cleavage_freq"])
s1 = stats.tstd(df_train["transformed_cleavage_freq"])
m2 = 0
s2 = math.sqrt(2)
df_train["scaled_cleavage_freq"] = m2 + (df_train["transformed_cleavage_freq"] - m1) * (s2/s1)

# Manually transform outliers
df_train["scaled_cleavage_freq"] = np.where(df_train["scaled_cleavage_freq"] >2*s2, 2*s2,df_train["scaled_cleavage_freq"])
df_train["scaled_cleavage_freq"] = np.where(df_train["scaled_cleavage_freq"] < -2*s2, -2*s2,df_train["scaled_cleavage_freq"])
low_cleavage["scaled_cleavage_freq"] = -2*s2

# 
df_train = df_train.drop(columns=["transformed_cleavage_freq"])
df_train = pd.concat([df_train, low_cleavage])

In [ ]:
plt.hist(df_train["scaled_cleavage_freq"])

In [ ]:
#df_train.to_pickle("../data/df_train.pkl")

In [ ]:
#df_test.to_pickle("../data/df_test.pkl")

In [9]:
df_train = df_train["target_sequence", "grna_target_sequence", "target_strand", "grna_strand", "energy_1", "energy_2", "energy_3", "energy_4", "energy_5", "study_name", "whole_genome", "delivery_mode"]

KeyError: ('target_sequence', 'grna_target_sequence', 'target_strand', 'grna_strand', 'energy_1', 'energy_2', 'energy_3', 'energy_4', 'energy_5', 'study_name', 'whole_genome', 'delivery_mode')

In [10]:
def select(df, params):
    df = df[params]
    df[params] = df[params].convert_dtypes()
    return df

In [13]:
params = ["target_sequence", "grna_target_sequence", "target_strand", "grna_target_strand", "energy_1", "energy_2", "energy_3", "energy_4", "energy_5", "study_name", "whole_genome", "delivery_mode"]

In [16]:
select(df_train, params)

/home/megan/.local/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,target_sequence,grna_target_sequence,target_strand,grna_target_strand,energy_1,energy_2,energy_3,energy_4,energy_5,study_name,whole_genome,delivery_mode
0,ACCCCCCCCAACCCCGCCTCGGC,GACCCCCTCCACCCCGCCTCCGG,+,-,26.355,0.000000,8.809075,0.0000,26.355,Tsai_circle,1,0
1,AAGAGGAGGGAGATTGTTCCTGG,GGGTGGGGGGAGTTTGCTCCTGG,-,-,12.735,-4.939260,-4.939260,16.3350,16.335,Tsai_circle,1,0
2,GTGATAAGTGGAATTGCCATGTGAG,GTGATAAGTGGAATGCCATGTGG,+,+,-12.695,-42.815277,-47.572529,-9.1755,-10.195,Finkelstein,0,2
3,AGCACTGTGGATGGAGTTGGAGG,GGCACTGCGGCTGGAGGTGGGGG,+,+,17.605,5.982632,5.982632,21.1050,21.105,Tsai_circle,1,0
4,GTGATAAGATGGAATGCCATGTGGG,GTGATAAGTGGAATGCCATGTGG,+,+,-13.315,-48.090126,-48.090126,-10.8150,-10.815,Finkelstein,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
20500,GTGGATAAGTGGAACTGCCATGTGG,GTGATAAGTGGAATGCCATGTGG,+,+,-2.095,-26.843519,-26.843519,0.4050,0.405,Finkelstein,0,2
20501,GACGCATAAAGATGAGACGCTTC,GACGCATAAAGATGAGACGCTGG,+,+,28.700,0.000000,21.484870,0.0000,28.700,Finkelstein,0,2
20502,GTGATAAAGTGGAATCGCCATGTGG,GTGATAAGTGGAATGCCATGTGG,+,+,-3.700,-29.040044,-29.040044,-1.2000,-1.200,Finkelstein,0,2
20503,TCTCCCCGCCCCCTCGCCTCTGG,GACCCCCTCCACCCCGCCTCCGG,-,-,17.935,-6.524068,-6.524068,17.9350,17.935,Tsai_circle,1,0
